# WechatSogou

### algorithm:
1. import module
2. get gzh historical articles
3. save the html to local disk
4. insert into database or update sids

## 1. import modules

In [14]:
import wechatsogou
import random
import requests
import datetime
import time
from lxml.etree import XMLSyntaxError
from requests.exceptions import ConnectionError
from pyquery import PyQuery as pq
import os
import pathlib
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
# 直连
ws_api = wechatsogou.WechatSogouAPI()

# 验证码输入错误的重试次数，默认为1
ws_api = wechatsogou.WechatSogouAPI(captcha_break_time=3)

# 所有requests库的参数都能在这用
# 如 配置代理，代理列表中至少需包含1个 HTTPS 协议的代理, 并确保代理可用
ws_api = wechatsogou.WechatSogouAPI(proxies={
    "http": "127.0.0.1:8888",
    "https": "127.0.0.1:8888",
})

# 如 设置超时
ws_api = wechatsogou.WechatSogouAPI(timeout=1)


In [3]:
headers = {
    'Cookie': """SUID=2F2E40613921940A000000005C7C8A68; SUV=0054C7AB61402E2F5C7C8A6C4E9DE485; IPLOC=CN3100; UM_distinctid=16af8a3d10b1c6-00075670c610bf-e353165-1fa400-16af8a3d10c66e; ABTEST=5|1560408316|v1; JSESSIONID=aaafTsCy_KK2le_X5lqRw; PHPSESSID=38sddn82q3fr3mihpq5s9j09j7; SNUID=A3091D2C494FC612B1C5C56E4A2CA7BE; sct=7""",
    'Host': 'weixin.sogou.com',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36'
}

In [4]:
ua = [
    "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
    "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
    "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
    "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
    "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
    "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
    "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
    "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
    "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
    "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
    "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
]


## get proxy tests

In [ ]:
proxies

In [ ]:
proxy = '117.91.132.172	9999'.replace('	',':')
proxy

In [ ]:
proxy_agent_url = "https://www.xicidaili.com/wn/"
proxies = {
                'http':'http://'+ proxy, #设置协议类型
                'https':'https://'+ '112.80.41.78:8888' #设置协议类型
    }
uachoice = random.choice(ua)
headers = {'Cookie': """SUID=2F2E40613921940A000000005C7C8A68; SUV=0054C7AB61402E2F5C7C8A6C4E9DE485; IPLOC=CN3100; UM_distinctid=16af8a3d10b1c6-00075670c610bf-e353165-1fa400-16af8a3d10c66e; ABTEST=5|1560408316|v1; JSESSIONID=aaafTsCy_KK2le_X5lqRw; PHPSESSID=38sddn82q3fr3mihpq5s9j09j7; SNUID=A3091D2C494FC612B1C5C56E4A2CA7BE; sct=7""",
"User-Agent": uachoice}
try:
    response = requests.get(proxy_agent_url, headers=headers, proxies=proxies)
    if response.status_code == 200:
        result = response.text
    else:
        raise KeyError
except ConnectionError:
    raise ConnectionError


In [ ]:
response

In [ ]:
soup = BeautifulSoup(result,'lxml')

In [ ]:
soup.find(id='body')

In [ ]:
soup.find(id='body').find_all()

In [ ]:
soup.find_all('tr')

In [ ]:
#获取代理
with open('ip.txt') as fp:
    proxy_pool = fp.readlines()
ls_proxy = []
for i in range(0, len(proxy_pool)):
    ip = proxy_pool[i].strip().split("\t")
    if ip[2] == 'HTTP':
        proxy_host = ip[0]+":"+ip[1]    
        ls_proxy.append(proxy_host)


In [ ]:
ls_proxy

## get articles

In [5]:
def get_proxy():
    http_url = "http://ip111.cn"
    proxy_temp = random.choice(proxy_pool)
    try:
        response = requests.get(http_url)
        if response.status_code == 200:
            return proxy_temp
        return None
    except ConnectionError:
        return None


In [ ]:
ws_api.get_gzh_info('南航青年志愿者')

In [ ]:
gzh = ws_api.search_gzh('付鹏的财经世界')

In [ ]:
list(gzh)

In [ ]:
articles = ws_api.search_article('付鹏的财经世界')

In [ ]:
list(articles)

In [ ]:
ws_api.get_sugg('财经')

### Search historical articles

In [6]:
proxy = None #将代理设为全局变量
max_count = 3 #最大请求次数

In [7]:
def get_html(url,count = 1):
    #打印一些调试信息
    print('Crawling:', url)
    print('Trying Count:', count)

    global proxy #引用全局变量
    if count >= max_count: #如果请求次数达到了上限
        print('Tried too many counts！')
        return None

    try :
        uachoice = random.choice(ua)
        headers = {'Cookie': """SUID=2F2E40613921940A000000005C7C8A68; SUV=0054C7AB61402E2F5C7C8A6C4E9DE485; IPLOC=CN3100; UM_distinctid=16af8a3d10b1c6-00075670c610bf-e353165-1fa400-16af8a3d10c66e; ABTEST=5|1560408316|v1; JSESSIONID=aaafTsCy_KK2le_X5lqRw; PHPSESSID=38sddn82q3fr3mihpq5s9j09j7; SNUID=A3091D2C494FC612B1C5C56E4A2CA7BE; sct=7""",
        "User-Agent": uachoice}
        if proxy:# 如果现在正在使用代理
            proxies = {
                'http':'http://'+ proxy #设置协议类型
            }
            response = requests.get(url, headers=headers, proxies=proxies, timeout=3) #使用有代理参数的请求
        else: #否则使用正常的请求
            response = requests.get(url, headers=headers, timeout=3)#禁止自动处理跳转
        if response.status_code == 200:
            print("请求成功！")
            return response.text
        if response.status_code == 302:
            # 需要代理
            print("302！")
            proxy = get_proxy()
            if proxy:
                print('Using Proxy', proxy)
                return get_html(url)
            else:
                print('Get Proxy Failed')
                return None
    except ConnectionError as e:
        print('Error Occurred', e.args)#打印错误信息
        proxy = get_proxy() #若失败，更换代理
        count += 1 #请求次数+1
        time.sleep(2)
        return get_html(url, count) #重试
    
    

In [22]:
# 解析详情页
def parse_detail(html):
    doc = pq(html)
    title = doc('.rich_media_title').text()
    content = doc('.rich_media_content').text()
    date = doc('#post-date').text()
    nickname = doc('#js_profile_qrcode > div > strong').text()
    wechat = doc('#js_profile_qrcode > div > p:nth-child(3) > span').text()
    return {
        'title':title,
        'content':content,
        'date':date,
        'nickname':nickname,
        'wechat':wechat
    }

In [8]:
with open('KOL.txt') as fp:
    ls_kol = fp.readlines()
    ls_kol = [x.replace('\n', '') for x in ls_kol]

In [9]:
ls_kol

['付鹏的财经世界',
 '长赢指数投资',
 '博策远略',
 '新全球资产配置',
 '沧海一土狗',
 '川流不息',
 'Trader.M的知乎文章',
 'StochasticWalker的知乎文章',
 '全球资产观察',
 '湖畔公寓28号',
 'Bridgewater - Daily Observations',
 '半夏投资',
 '因子投资那些事儿',
 '跨资产的因子配置',
 '宏观大类策略研究',
 'Chihiro Quantitative Research',
 '量化交易&宽客',
 '淳臻投资',
 'cfuwxd',
 '孟岩',
 '东京交易厅',
 '聚宽量化实验室',
 '饭统戴老板',
 '黑猫Q形态的知乎文章',
 '关关楼西',
 '高善文经济观察',
 'Moneycode',
 '全球经济每日漫谈',
 '月风投资笔记',
 '伦敦交易员',
 'babyquant谈量化金融',
 '期权策略交易',
 '金融学前沿论文速递',
 '李超宏观研究与资产配置',
 '人民币交易与研究',
 '筹码',
 '投研帮',
 'Pure Delta',
 '向小田',
 '【量化投资与机器学习】微信公众号',
 '量化投资大家学',
 '深圳天谷资产管理有限公司',
 '40秒',
 '岭峰资本']

loop in the list key of leaders:
get history
loop in the history article url
if there's author's directory

In [21]:
for au in tqdm(ls_kol):
    print('now: %s'%au)
    while True:
        # get latest 10 articles
        try:
            history = ws_api.get_gzh_article_by_history(au)
        except:
            pass
            print('failed to fetch historical articles.')
        else:
            break
        time.sleep(2)
        
    # get articles 
    if len(history) == 0:
        continue
    # make directory
    wechat_name = history['gzh']['wechat_name']
    export_dir = 'data/{0}/'.format(wechat_name)
    if not os.path.exists(export_dir):
        pathlib.Path(export_dir).mkdir(parents=True, exist_ok=True)
    for ci in history['article']:
        author = ci['author']
        title = ci['title']
        dt = ci['datetime']
        dt = datetime.datetime.fromtimestamp(
            int(dt)
        ).strftime('%Y_%m_%d')
        url = ci['content_url']
        print(title)
        # use requests
        title = title.replace('/', '')
        fpath = 'data/{0}/{1}_{2}_{3}.html'.format(wechat_name, dt, author, title)
        if not os.path.exists(fpath):
            article = get_html(url)
            if article:
                # export article
                with open(fpath, 'w') as fp:
                    fp.write(article)
        else:
            continue
        time.sleep(2)    
    time.sleep(2)



  0%|          | 0/44 [00:00<?, ?it/s]

now: 付鹏的财经世界
现在白银的问题是什么？历史给你演绎答案
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821303&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWIAHREzOUN6Rqy0XIqfFSCgWH9hmMtDHqnO4ASmKty5oKk-KzNFk1otX-A7UGlyTIbOaoM9uBzrlsXnis0nUnm9STqkxlGdZrQVQJwDKRGkbinAFb3Sdl4JbjP2vWQxULw8FTc*zJ8hPENBIu9boqdo=
Trying Count: 1
请求成功！
黄金的温故而知新日记：对比2016年
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821303&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWIAHREzOUN6Rqy0XIqfFSCgWH9hmMtDHqnO4ASmKty5oKk-KzNFk1otX-A7UGlyTIa0KK18eRF8*j4vD3laDnH0HmaqnbR2-*pjgtbPqaBBimC-GdHDxiI5Rvy*6hhbOfOr4LGjcAu-ccQFt0aevHxA=
Trying Count: 1
请求成功！
窘迫的央行-背后越来越庞大的负利率债券规模
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821303&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWIAHREzOUN6Rqy0XIqfFSCgWH9hmMtDHqnO4ASmKty5oKk-KzNFk1otX-A7UGlyTIXx9SAyaTaULquPhxHXURaF0wArBcuLoXJkOBjdOAngYuPi3mG5QmZnCJB954WrTbFOUn-r6vGx4NX8mpivM5ew=
Trying Count: 1
请求成功！
看似是美伊的事，实是五角大楼对白宫的倒逼
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821303&src=3&ver=1&signature=g6uT



  2%|▏         | 1/44 [00:23<16:49, 23.48s/it]

now: 长赢指数投资
2019年6月ETF计划（二）：买入两份
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821327&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWDxfKBe*HmxmVjC-Bj8Siti6Le2tbBxebjiLWpxJdbTXvJa799HZNxXgjtDTb8cAbY94P4BLHmgK*AEWKtWKbmh9-mdgC8nvyCYtgbR7KOtNF*Lpe6dSo0BamaB6o0t2lOLU1W7VXe1cOAQqnycEH2E=
Trying Count: 1
请求成功！
2019年6月ETF计划（一）：买入两份
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821327&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWDxfKBe*HmxmVjC-Bj8Siti6Le2tbBxebjiLWpxJdbTXvJa799HZNxXgjtDTb8cAbWdm*r3V1dg-ozRYBArJqNhKnYTeGzlQM0G3iegSci0JwTXbuTfCRA95gi3x6PcFc0NpLUdyi6N*nnsY3OkaNkk=
Trying Count: 1
请求成功！
波段策略.网格之三：网格策略进阶/2.0版
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821327&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWDxfKBe*HmxmVjC-Bj8Siti6Le2tbBxebjiLWpxJdbTXvJa799HZNxXgjtDTb8cAbTgmf4ah7Klu*5u3kBGVoP0PJnAf68cOoHKF8wByNxwAYcpRCVKJjsCFQ6McJNmbbmXQVX7idW3PeRau8Hh43C0=
Trying Count: 1
请求成功！
2019年5月ETF计划（二）：买入两份；卖出一份
Crawling: http://mp.weixin.qq.com/s?timestamp=1561821327&src=3&ver=1&signatu



  5%|▍         | 2/44 [00:47<16:27, 23.52s/it]

...



 95%|█████████▌| 42/44 [16:26<00:58, 29.32s/it]

now: 40秒
这小子正在和马斯克一起拯救世界，顺便统治一下硅谷
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822290&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWAN8U3CiFUVFplOG8twgVAln3TkAVS3*ef2*hoe5ML0AxfkdAicUowDo5ou93TsatfnNBzAKS1RRSksnkWdhh3Z2wbDaDzLEan8daw6SELiAnjFQ1TrbLJb6vtGCweVHmcRsLZocMLdYzMkBp6nxlhU=
Trying Count: 1
请求成功！
这是人类历史上最贵的一场撕逼大战 | 华尔街黑历史（六）
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822290&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWAN8U3CiFUVFplOG8twgVAln3TkAVS3*ef2*hoe5ML0AxfkdAicUowDo5ou93TsatR3BlHs4XCuIoiOrafJOxc7VbwRiS1ickpmFpDwmD9cjTSPNPxfhmRMIrz6uni5SSeZXukDdtGLt-LIydvVNjwU=
Trying Count: 1
请求成功！
就是这小子，把卡扎菲给忽悠瘸了 | 华尔街黑历史（五）
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822290&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWAN8U3CiFUVFplOG8twgVAln3TkAVS3*ef2*hoe5ML0AxfkdAicUowDo5ou93TsatTJbZabXdgvrEd*hAR4T7gsgEcAKw3ty0KiCPhCBF5lfY7L0Vv30lGmlsg6JznVEwG9rf6s-2u6qhU35v1zVcB8=
Trying Count: 1
请求成功！
驯服贝克汉姆和C罗的人
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822290&src=3&ver=1&signat



 98%|█████████▊| 43/44 [16:51<00:27, 27.74s/it]

now: 岭峰资本
关于《信用的秘密》一文的说明
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822314&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWGE5HowTxeae2IIqB65RozVN-EUCghI5QscHQ034zVHSytW7g*YNDDXAN4P2KEYq5d4hTdc*HfCwdxcrpAx4X9SXgHmKA4mx7Nz5LQISO7T0QmZbqWB9IJtqGXYmiwHvvgxpIxKMkqCSrd6DR2-2AHQ=
Trying Count: 1
请求成功！
暂停公众号文章
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822314&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWGE5HowTxeae2IIqB65RozVN-EUCghI5QscHQ034zVHSytW7g*YNDDXAN4P2KEYq5aL1KquGfUZDcQvnpHsMjU53Er-qyFuwnLgSGwOD77VOtKPqX5gmnhd5l74WUBqTIK6xfjitLoIQ6WyRuQuZq0k=
Trying Count: 1
请求成功！
信用的秘密
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822314&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWGE5HowTxeae2IIqB65RozVN-EUCghI5QscHQ034zVHSytW7g*YNDDXAN4P2KEYq5SW1CR6XwLwlsD9rQhMuzz9Zw71BwABNWz0mSOfNw1YarGULVzSFQnl08ZpGKF8YfMrvz-9wGaRAbZkDsqSIA04=
Trying Count: 1
请求成功！
保皮？保毛？还是保命？
Crawling: http://mp.weixin.qq.com/s?timestamp=1561822314&src=3&ver=1&signature=g6uT2E0G36oPT8EKvdefWGE5HowTxeae2IIqB65RozVN-EUC



100%|██████████| 44/44 [17:14<00:00, 26.60s/it]

In [ ]:
list(history)

In [ ]:
history['gzh']